# Death - weighted SVM - period 1

In [1]:
#import dependencies
import pandas as pd
import numpy as np

In [2]:
#read in data set
df = pd.read_csv("Resources/period1_final_data.csv")
df.head(20)

,RANDID,SEX,TOTCHOL,AGE,SYSBP,DIABP,CURSMOKE,CIGPDAY,BMI,DIABETES,...,CVD,HYPERTEN,TIMEAP,TIMEMI,TIMEMIFC,TIMECHD,TIMESTRK,TIMECVD,TIMEDTH,TIMEHYP
0,2448,1,195.0,39,106.0,70.0,0,0.0,26.97,0,...,1,0,8766,6438,6438,6438,8766,6438,8766,8766
1,6238,2,250.0,46,121.0,81.0,0,0.0,28.73,0,...,0,0,8766,8766,8766,8766,8766,8766,8766,8766
2,9428,1,245.0,48,127.5,80.0,1,20.0,25.34,0,...,0,0,8766,8766,8766,8766,8766,8766,8766,8766
3,10552,2,225.0,61,150.0,95.0,1,30.0,28.58,0,...,1,1,2956,2956,2956,2956,2089,2089,2956,0
4,11252,2,285.0,46,130.0,84.0,1,23.0,23.10,0,...,0,1,8766,8766,8766,8766,8766,8766,8766,4285
5,11263,2,228.0,43,180.0,110.0,0,0.0,30.30,0,...,1,1,8766,8766,5719,5719,8766,5719,8766,0
6,12629,2,205.0,63,138.0,71.0,0,0.0,33.11,0,...,0,1,373,8766,8766,373,8766,8766,8766,2212
7,12806,2,313.0,45,100.0,71.0,1,20.0,21.68,0,...,0,1,8766,8766,8766,8766,8766,8766,8766,8679
8,14367,1,260.0,52,141.5,89.0,0,0.0,26.36,0,...,0,1,8766,8766,8766,8766,8766,8766,8766,0
9,16365,1,225.0,43,162.0,107.0,1,30.0,23.61,0,...,0,1,8766,8766,8766,8766,8766,8766,8766,0


In [3]:
death_df = df.set_index('RANDID')
#Remove columns not involved in analysis
death_df = death_df[['SEX', 'TOTCHOL', 'AGE', 'SYSBP', 'DIABP', 'CURSMOKE', 'CIGPDAY', 'BMI',
       'DIABETES', 'BPMEDS', 'HEARTRTE', 'GLUCOSE', 'PREVCHD', 'PREVAP',
       'PREVMI', 'PREVSTRK', 'PREVHYP', 'DEATH']]
death_df.head()

,SEX,TOTCHOL,AGE,SYSBP,DIABP,CURSMOKE,CIGPDAY,BMI,DIABETES,BPMEDS,HEARTRTE,GLUCOSE,PREVCHD,PREVAP,PREVMI,PREVSTRK,PREVHYP,DEATH
RANDID,,,,,,,,,,,,,,,,,,
2448,1,195.0,39,106.0,70.0,0,0.0,26.97,0,0.0,80.0,77.0,0,0,0,0,0,0
6238,2,250.0,46,121.0,81.0,0,0.0,28.73,0,0.0,95.0,76.0,0,0,0,0,0,0
9428,1,245.0,48,127.5,80.0,1,20.0,25.34,0,0.0,75.0,70.0,0,0,0,0,0,0
10552,2,225.0,61,150.0,95.0,1,30.0,28.58,0,0.0,65.0,103.0,0,0,0,0,1,1
11252,2,285.0,46,130.0,84.0,1,23.0,23.10,0,0.0,85.0,85.0,0,0,0,0,0,0


In [4]:
target = death_df["DEATH"]
target_names = ["negative", "positive"]

data = death_df.drop("DEATH", axis=1)
feature_names = data.columns
data.head()

,SEX,TOTCHOL,AGE,SYSBP,DIABP,CURSMOKE,CIGPDAY,BMI,DIABETES,BPMEDS,HEARTRTE,GLUCOSE,PREVCHD,PREVAP,PREVMI,PREVSTRK,PREVHYP
RANDID,,,,,,,,,,,,,,,,,
2448,1,195.0,39,106.0,70.0,0,0.0,26.97,0,0.0,80.0,77.0,0,0,0,0,0
6238,2,250.0,46,121.0,81.0,0,0.0,28.73,0,0.0,95.0,76.0,0,0,0,0,0
9428,1,245.0,48,127.5,80.0,1,20.0,25.34,0,0.0,75.0,70.0,0,0,0,0,0
10552,2,225.0,61,150.0,95.0,1,30.0,28.58,0,0.0,65.0,103.0,0,0,0,0,1
11252,2,285.0,46,130.0,84.0,1,23.0,23.10,0,0.0,85.0,85.0,0,0,0,0,0


In [6]:
death_df["DEATH"].value_counts()

0    2552
1    1375
Name: DEATH, dtype: int64

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=49)

In [8]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScater model and fit it to the training data

X_scaler = StandardScaler().fit(X_train)

In [9]:
# Transform the training and testing data using the X_scaler and y_scaler models

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
# Support vector machine linear classifier
from sklearn.svm import SVC 
model = SVC(kernel='linear', class_weight={1:2})

In [11]:
# Create the GridSearch estimator along with a parameter object containing the values to adjust
# Adjust `C` with values of 1, 5, and 10. Adjust `gamma` using .0001, 0.001, and 0.01
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.1, 0.5, 1, 5, 10],
              'gamma': [0.0001, 0.001, 0.01]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [12]:
# Fit the model using the grid search estimator. 
# This will take the SVC model and try each combination of parameters
grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 15 candidates, totalling 75 fits
[CV] C=0.1, gamma=0.0001 .............................................
[CV] ................. C=0.1, gamma=0.0001, score=0.722, total=   0.1s
[CV] C=0.1, gamma=0.0001 .............................................
[CV] ................. C=0.1, gamma=0.0001, score=0.688, total=   0.1s
[CV] C=0.1, gamma=0.0001 .............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s


[CV] ................. C=0.1, gamma=0.0001, score=0.772, total=   0.1s
[CV] C=0.1, gamma=0.0001 .............................................
[CV] ................. C=0.1, gamma=0.0001, score=0.727, total=   0.1s
[CV] C=0.1, gamma=0.0001 .............................................
[CV] ................. C=0.1, gamma=0.0001, score=0.754, total=   0.1s
[CV] C=0.1, gamma=0.001 ..............................................
[CV] .................. C=0.1, gamma=0.001, score=0.722, total=   0.1s
[CV] C=0.1, gamma=0.001 ..............................................
[CV] .................. C=0.1, gamma=0.001, score=0.688, total=   0.1s
[CV] C=0.1, gamma=0.001 ..............................................
[CV] .................. C=0.1, gamma=0.001, score=0.772, total=   0.1s
[CV] C=0.1, gamma=0.001 ..............................................
[CV] .................. C=0.1, gamma=0.001, score=0.727, total=   0.1s
[CV] C=0.1, gamma=0.001 ..............................................
[CV] .

[CV] .................. C=10, gamma=0.0001, score=0.725, total=   0.7s
[CV] C=10, gamma=0.0001 ..............................................
[CV] .................. C=10, gamma=0.0001, score=0.681, total=   0.7s
[CV] C=10, gamma=0.0001 ..............................................
[CV] .................. C=10, gamma=0.0001, score=0.772, total=   0.7s
[CV] C=10, gamma=0.0001 ..............................................
[CV] .................. C=10, gamma=0.0001, score=0.727, total=   0.6s
[CV] C=10, gamma=0.0001 ..............................................
[CV] .................. C=10, gamma=0.0001, score=0.756, total=   0.8s
[CV] C=10, gamma=0.001 ...............................................
[CV] ................... C=10, gamma=0.001, score=0.725, total=   0.6s
[CV] C=10, gamma=0.001 ...............................................
[CV] ................... C=10, gamma=0.001, score=0.681, total=   0.7s
[CV] C=10, gamma=0.001 ...............................................
[CV] .

[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:   23.3s finished


GridSearchCV(estimator=SVC(class_weight={1: 2}, kernel='linear'),
             param_grid={'C': [0.1, 0.5, 1, 5, 10],
                         'gamma': [0.0001, 0.001, 0.01]},
             verbose=3)

In [13]:
# List the best parameters for this dataset
print(grid.best_params_)

{'C': 0.1, 'gamma': 0.0001}


In [14]:
# List the best score
print(grid.best_score_)

0.732427843803056


In [15]:
# Make predictions with the hypertuned model
predictions = grid.predict(X_test_scaled)

In [16]:
# Calculate classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions,
                            target_names=target_names))

              precision    recall  f1-score   support

    negative       0.80      0.74      0.77       630
    positive       0.59      0.68      0.63       352

    accuracy                           0.72       982
   macro avg       0.70      0.71      0.70       982
weighted avg       0.73      0.72      0.72       982



In [17]:
cvd_predictions = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
cvd_predictions

,Prediction,Actual
0,0,0
1,0,1
2,0,0
3,0,0
4,0,0
...,...,...
977,1,1
978,1,1
979,0,0
980,0,1


In [18]:
p0a0 = cvd_predictions.loc[(cvd_predictions['Prediction']==0) & (cvd_predictions['Actual']==0)]
p1a1 = cvd_predictions.loc[(cvd_predictions['Prediction']==1) & (cvd_predictions['Actual']==1)]
p0a1 = cvd_predictions.loc[(cvd_predictions['Prediction']==0) & (cvd_predictions['Actual']==1)]
p1a0 = cvd_predictions.loc[(cvd_predictions['Prediction']==1) & (cvd_predictions['Actual']==0)]

print(f"Event prediction = 0 and actual = 0:  {len(p0a0)}")
print(f"Event prediction = 1 and actual = 1:  {len(p1a1)}")
print(f"Event prediction = 0 and actual = 1:  {len(p0a1)}")
print(f"Event prediction = 1 and actual = 0:  {len(p1a0)}")

Event prediction = 0 and actual = 0:  466
Event prediction = 1 and actual = 1:  239
Event prediction = 0 and actual = 1:  113
Event prediction = 1 and actual = 0:  164


## Separated by Gender

In [19]:
#filter df by gender (males = 1, females = 2)
male_death = death_df.loc[(death_df["SEX"] ==1)]
female_death = death_df.loc[(death_df["SEX"] ==2)]

male_death_df = male_death.drop('SEX', axis=1)
female_death_df = female_death.drop('SEX', axis=1)

In [20]:
male_target = male_death_df["DEATH"]
male_target_names = ["negative", "positive"]

female_target = female_death_df["DEATH"]
female_target_names = ["negative", "positive"]

male_data = male_death_df.drop("DEATH",axis=1)
male_feature_names = male_data.columns

female_data = female_death_df.drop("DEATH", axis=1)
female_feature_names = female_data.columns

In [21]:
# Split the data using train_test_split - males
mX_train, mX_test, my_train, my_test = train_test_split(male_data, male_target, random_state=49)

In [22]:
# Create a StandardScater model and fit it to the training data

mX_scaler = StandardScaler().fit(mX_train)

# Transform the training and testing data using the X_scaler and y_scaler models

mX_train_scaled = mX_scaler.transform(mX_train)
mX_test_scaled = mX_scaler.transform(mX_test)

In [23]:
male_death_df["DEATH"].value_counts()

0    1009
1     773
Name: DEATH, dtype: int64

In [24]:
# Support vector machine linear classifier

m_model = SVC(kernel='linear')

# Create the GridSearch estimator along with a parameter object containing the values to adjust

m_param_grid = {'C': [0.1, 0.5, 1, 5, 10],
              'gamma': [0.0001, 0.001, 0.01]}
m_grid = GridSearchCV(m_model, m_param_grid, verbose=3)

In [25]:
# Fit the model using the grid search estimator. 

m_grid.fit(mX_train_scaled, my_train)

Fitting 5 folds for each of 15 candidates, totalling 75 fits
[CV] C=0.1, gamma=0.0001 .............................................
[CV] ................. C=0.1, gamma=0.0001, score=0.743, total=   0.0s
[CV] C=0.1, gamma=0.0001 .............................................
[CV] ................. C=0.1, gamma=0.0001, score=0.704, total=   0.0s
[CV] C=0.1, gamma=0.0001 .............................................
[CV] ................. C=0.1, gamma=0.0001, score=0.757, total=   0.0s
[CV] C=0.1, gamma=0.0001 .............................................
[CV] ................. C=0.1, gamma=0.0001, score=0.742, total=   0.0s
[CV] C=0.1, gamma=0.0001 .............................................
[CV] ................. C=0.1, gamma=0.0001, score=0.772, total=   0.0s
[CV] C=0.1, gamma=0.001 ..............................................
[CV] .................. C=0.1, gamma=0.001, score=0.743, total=   0.0s
[CV] C=0.1, gamma=0.001 ..............................................
[CV] ...........

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV] .................. C=0.1, gamma=0.001, score=0.772, total=   0.0s
[CV] C=0.1, gamma=0.01 ...............................................
[CV] ................... C=0.1, gamma=0.01, score=0.743, total=   0.0s
[CV] C=0.1, gamma=0.01 ...............................................
[CV] ................... C=0.1, gamma=0.01, score=0.704, total=   0.0s
[CV] C=0.1, gamma=0.01 ...............................................
[CV] ................... C=0.1, gamma=0.01, score=0.757, total=   0.0s
[CV] C=0.1, gamma=0.01 ...............................................
[CV] ................... C=0.1, gamma=0.01, score=0.742, total=   0.0s
[CV] C=0.1, gamma=0.01 ...............................................
[CV] ................... C=0.1, gamma=0.01, score=0.772, total=   0.0s
[CV] C=0.5, gamma=0.0001 .............................................
[CV] ................. C=0.5, gamma=0.0001, score=0.743, total=   0.0s
[CV] C=0.5, gamma=0.0001 .............................................
[CV] .

[CV] ................... C=10, gamma=0.001, score=0.760, total=   0.1s
[CV] C=10, gamma=0.001 ...............................................
[CV] ................... C=10, gamma=0.001, score=0.745, total=   0.1s
[CV] C=10, gamma=0.001 ...............................................
[CV] ................... C=10, gamma=0.001, score=0.775, total=   0.1s
[CV] C=10, gamma=0.01 ................................................
[CV] .................... C=10, gamma=0.01, score=0.743, total=   0.1s
[CV] C=10, gamma=0.01 ................................................
[CV] .................... C=10, gamma=0.01, score=0.704, total=   0.1s
[CV] C=10, gamma=0.01 ................................................
[CV] .................... C=10, gamma=0.01, score=0.760, total=   0.1s
[CV] C=10, gamma=0.01 ................................................
[CV] .................... C=10, gamma=0.01, score=0.745, total=   0.1s
[CV] C=10, gamma=0.01 ................................................
[CV] .

[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:    4.3s finished


GridSearchCV(estimator=SVC(kernel='linear'),
             param_grid={'C': [0.1, 0.5, 1, 5, 10],
                         'gamma': [0.0001, 0.001, 0.01]},
             verbose=3)

In [26]:
# List the best parameters for this dataset
print(m_grid.best_params_)

{'C': 1, 'gamma': 0.0001}


In [27]:
# Make predictions with the hypertuned model - male
m_predictions = m_grid.predict(mX_test_scaled)

# Calculate classification report
from sklearn.metrics import classification_report
print(classification_report(my_test, m_predictions,
                            target_names=male_target_names))

              precision    recall  f1-score   support

    negative       0.77      0.83      0.80       261
    positive       0.73      0.65      0.69       185

    accuracy                           0.76       446
   macro avg       0.75      0.74      0.74       446
weighted avg       0.75      0.76      0.75       446



In [28]:
m_death_predictions = pd.DataFrame({"Prediction": m_predictions, "Actual": my_test})

mp0a0 = m_death_predictions.loc[(m_death_predictions['Prediction']==0) & (m_death_predictions['Actual']==0)]
mp1a1 = m_death_predictions.loc[(m_death_predictions['Prediction']==1) & (m_death_predictions['Actual']==1)]
mp0a1 = m_death_predictions.loc[(m_death_predictions['Prediction']==0) & (m_death_predictions['Actual']==1)]
mp1a0 = m_death_predictions.loc[(m_death_predictions['Prediction']==1) & (m_death_predictions['Actual']==0)]

print(f"Event prediction = 0 and actual = 0:  {len(mp0a0)}")
print(f"Event prediction = 1 and actual = 1:  {len(mp1a1)}")
print(f"Event prediction = 0 and actual = 1:  {len(mp0a1)}")
print(f"Event prediction = 1 and actual = 0:  {len(mp1a0)}")

Event prediction = 0 and actual = 0:  217
Event prediction = 1 and actual = 1:  120
Event prediction = 0 and actual = 1:  65
Event prediction = 1 and actual = 0:  44


In [29]:
female_death_df["DEATH"].value_counts()

0    1543
1     602
Name: DEATH, dtype: int64

In [30]:
# Split the data using train_test_split - females
fX_train, fX_test, fy_train, fy_test = train_test_split(female_data, female_target, random_state=49)

In [31]:
# Create a StandardScater model and fit it to the training data - females

fX_scaler = StandardScaler().fit(fX_train)

# Transform the training and testing data using the X_scaler and y_scaler models

fX_train_scaled = fX_scaler.transform(fX_train)
fX_test_scaled = fX_scaler.transform(fX_test)

In [32]:
# Support vector machine linear classifier - female

f_model = SVC(kernel='linear', class_weight={1:2.5})

# Create the GridSearch estimator along with a parameter object containing the values to adjust

f_param_grid = {'C': [0.1, 0.5, 1, 5, 10],
              'gamma': [0.0001, 0.001, 0.01]}
f_grid = GridSearchCV(f_model, f_param_grid, verbose=3)

In [33]:
# Fit the model using the grid search estimator. 

f_grid.fit(fX_train_scaled, fy_train)

Fitting 5 folds for each of 15 candidates, totalling 75 fits
[CV] C=0.1, gamma=0.0001 .............................................
[CV] ................. C=0.1, gamma=0.0001, score=0.776, total=   0.0s
[CV] C=0.1, gamma=0.0001 .............................................
[CV] ................. C=0.1, gamma=0.0001, score=0.724, total=   0.0s
[CV] C=0.1, gamma=0.0001 .............................................
[CV] ................. C=0.1, gamma=0.0001, score=0.699, total=   0.0s
[CV] C=0.1, gamma=0.0001 .............................................
[CV] ................. C=0.1, gamma=0.0001, score=0.735, total=   0.0s
[CV] C=0.1, gamma=0.0001 .............................................
[CV] ................. C=0.1, gamma=0.0001, score=0.701, total=   0.0s
[CV] C=0.1, gamma=0.001 ..............................................
[CV] .................. C=0.1, gamma=0.001, score=0.776, total=   0.0s
[CV] C=0.1, gamma=0.001 ..............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV] .................. C=0.1, gamma=0.001, score=0.724, total=   0.0s
[CV] C=0.1, gamma=0.001 ..............................................
[CV] .................. C=0.1, gamma=0.001, score=0.699, total=   0.0s
[CV] C=0.1, gamma=0.001 ..............................................
[CV] .................. C=0.1, gamma=0.001, score=0.735, total=   0.0s
[CV] C=0.1, gamma=0.001 ..............................................
[CV] .................. C=0.1, gamma=0.001, score=0.701, total=   0.0s
[CV] C=0.1, gamma=0.01 ...............................................
[CV] ................... C=0.1, gamma=0.01, score=0.776, total=   0.0s
[CV] C=0.1, gamma=0.01 ...............................................
[CV] ................... C=0.1, gamma=0.01, score=0.724, total=   0.0s
[CV] C=0.1, gamma=0.01 ...............................................
[CV] ................... C=0.1, gamma=0.01, score=0.699, total=   0.0s
[CV] C=0.1, gamma=0.01 ...............................................
[CV] .

[CV] .................. C=10, gamma=0.0001, score=0.698, total=   0.3s
[CV] C=10, gamma=0.001 ...............................................
[CV] ................... C=10, gamma=0.001, score=0.780, total=   0.4s
[CV] C=10, gamma=0.001 ...............................................
[CV] ................... C=10, gamma=0.001, score=0.717, total=   0.2s
[CV] C=10, gamma=0.001 ...............................................
[CV] ................... C=10, gamma=0.001, score=0.693, total=   0.2s
[CV] C=10, gamma=0.001 ...............................................
[CV] ................... C=10, gamma=0.001, score=0.732, total=   0.3s
[CV] C=10, gamma=0.001 ...............................................
[CV] ................... C=10, gamma=0.001, score=0.698, total=   0.2s
[CV] C=10, gamma=0.01 ................................................
[CV] .................... C=10, gamma=0.01, score=0.780, total=   0.4s
[CV] C=10, gamma=0.01 ................................................
[CV] .

[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:    8.7s finished


GridSearchCV(estimator=SVC(class_weight={1: 2.5}, kernel='linear'),
             param_grid={'C': [0.1, 0.5, 1, 5, 10],
                         'gamma': [0.0001, 0.001, 0.01]},
             verbose=3)

In [34]:
# List the best parameters for this dataset
print(f_grid.best_params_)

{'C': 0.1, 'gamma': 0.0001}


In [35]:
# Make predictions with the hypertuned model - female
f_predictions = f_grid.predict(fX_test_scaled)

# Calculate classification report
from sklearn.metrics import classification_report
print(classification_report(fy_test, f_predictions,
                            target_names=female_target_names))

              precision    recall  f1-score   support

    negative       0.82      0.71      0.76       387
    positive       0.45      0.60      0.51       150

    accuracy                           0.68       537
   macro avg       0.63      0.66      0.64       537
weighted avg       0.72      0.68      0.69       537



In [36]:
f_death_predictions = pd.DataFrame({"Prediction": f_predictions, "Actual": fy_test})

fp0a0 = f_death_predictions.loc[(f_death_predictions['Prediction']==0) & (f_death_predictions['Actual']==0)]
fp1a1 = f_death_predictions.loc[(f_death_predictions['Prediction']==1) & (f_death_predictions['Actual']==1)]
fp0a1 = f_death_predictions.loc[(f_death_predictions['Prediction']==0) & (f_death_predictions['Actual']==1)]
fp1a0 = f_death_predictions.loc[(f_death_predictions['Prediction']==1) & (f_death_predictions['Actual']==0)]

print(f"Event prediction = 0 and actual = 0:  {len(fp0a0)}")
print(f"Event prediction = 1 and actual = 1:  {len(fp1a1)}")
print(f"Event prediction = 0 and actual = 1:  {len(fp0a1)}")
print(f"Event prediction = 1 and actual = 0:  {len(fp1a0)}")

Event prediction = 0 and actual = 0:  275
Event prediction = 1 and actual = 1:  90
Event prediction = 0 and actual = 1:  60
Event prediction = 1 and actual = 0:  112
